In [ ]:
import torch
from torch.utils.data import DataLoader
import albumentations as A
from datasets import GTA5, CityScapes
from models.deeplabv2.deeplabv2 import get_deeplab_v2
from models.bisenet.build_bisenet import BiSeNet
from training import train
from model_evaluation import save_results
from visualization import  plot_loss, plot_mIoU, plot_IoU
from config import BATCH_SIZE, NC, NUM_WORKERS, CITYSCAPE_SIZE, GTA5_SIZE, EPOCHS, CITYSCAPES_MEAN, CITYSCAPES_STD, GTA5_MEAN, GTA5_STD, augmentations
torch.cuda.manual_seed(42)

device = 'cuda' if torch.cuda.is_available() else 'cpu'



In [ ]:

transforms = A.Compose([
    A.Resize(CITYSCAPE_SIZE[0], CITYSCAPE_SIZE[1]),
])
transforms_GTA5 = A.Compose([
        A.Resize(GTA5_SIZE[0], GTA5_SIZE[1]),


])

cityscapes_train_dataset = CityScapes('./Cityscapes', 'train', transform=transforms)
cityscapes_test_dataset = CityScapes('./Cityscapes', 'val', transform=transforms)
GTA5_dataset = GTA5('./GTA5', transform=transforms_GTA5)

cityscapes_train_dataloader = DataLoader(cityscapes_train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
cityscapes_test_dataloader = DataLoader(cityscapes_test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
GTA5_dataloader = DataLoader(GTA5_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
# cityscapes_mean, city_scapes_std = calculate_mean_std(cityscapes_train_dataloader)
# GTA5_mean, GTA5_std = calculate_mean_std(GTA5_dataloader)

len(cityscapes_train_dataloader)

## 2.1 DeepLabV2

In [ ]:

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=255)
init_lr = 2.5e-4
model_deeplab = get_deeplab_v2(num_classes=19, pretrain=True, pretrain_model_path='./models/deeplab_resnet_pretrained_imagenet.pth').to(device)
if device == 'cuda' and torch.cuda.device_count() > 1:
    model_deeplab = torch.nn.DataParallel(model_deeplab).to(device)
optimizer_deeplab = torch.optim.SGD(model_deeplab.parameters(), lr=init_lr, momentum=0.9, weight_decay=5e-4)

deeplab_result = train(model = model_deeplab,
                       optimizer = optimizer_deeplab,
                       loss_fn = loss_fn, 
                       train_dataloader = cityscapes_train_dataloader,
                       validation_dataloader = cityscapes_test_dataloader, 
                       epochs = EPOCHS,
                       device = device,
                       model_name = "deeplab",
                       class_number= NC)

In [ ]:
save_results(model_deeplab, deeplab_result, "deeplab_performance_metrics_2_1", height=CITYSCAPE_SIZE[0], width=CITYSCAPE_SIZE[1], iterations=100, device=device)
plot_loss(deeplab_result, "DeepLabV2", "step2_1", "CityScapes", "CityScapes")
plot_mIoU(deeplab_result, "DeepLabV2", "step2_1", "CityScapes", "CityScapes")
plot_IoU(deeplab_result, "DeepLabV2", "step2_1", "CityScapes", "CityScapes")

torch.save(model_deeplab.state_dict(), "./checkpoints/deeplab_model_2_1.pth")

## 2.2 BiseNet


In [ ]:
init_lr = 2.5e-4
model_bisenet = BiSeNet(NC, 'resnet18').to(device)
if device == 'cuda' and torch.cuda.device_count() > 1:
    model_bisenet = torch.nn.DataParallel(model_bisenet).to(device)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=255)
optimizer_bisenet = torch.optim.Adam(model_bisenet.parameters(), lr=init_lr)

bisenet_result_2_2 = train(model = model_bisenet,
                           optimizer = optimizer_bisenet,
                           loss_fn = loss_fn, 
                           train_dataloader = cityscapes_train_dataloader,
                           validation_dataloader = cityscapes_test_dataloader, 
                           epochs = EPOCHS,
                           device = device,
                           model_name = "bisenet",
                           class_number= NC)



In [ ]:

import importlib
import model_evaluation
importlib.reload(model_evaluation)
from model_evaluation import save_results
model_bisenet.eval()
save_results(model_bisenet, bisenet_result_2_2, "bisenet_adam_performance_metrics_2_2", height=CITYSCAPE_SIZE[0], width=CITYSCAPE_SIZE[1], iterations=100)
plot_loss(bisenet_result_2_2, "BiSeNet", "step2_2_adam", "CityScapes", "CityScapes")
plot_mIoU(bisenet_result_2_2, "BiSeNet", "step2_2_adam", "CityScapes", "CityScapes")
plot_IoU(bisenet_result_2_2, "BiSeNet", "step2_2_adam", "CityScapes", "CityScapes")
torch.save(model_bisenet.state_dict(), "./checkpoints/bisenet_adam_model_2_2.pth")



## 3.1 Bisenet

In [ ]:
model_bisenet_3_1 = BiSeNet(19, 'resnet18').to(device)
if device == 'cuda' and torch.cuda.device_count() > 1:
    model_bisenet_3_1 = torch.nn.DataParallel(model_bisenet_3_1).to(device)
init_lr = 2.5e-4
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=255)
optimizer_bisenet_3_1 = torch.optim.Adam(model_bisenet_3_1.parameters(), lr=init_lr)
bisenet_result_3_1 = train(model=model_bisenet_3_1,
                           optimizer=optimizer_bisenet_3_1,
                           loss_fn=loss_fn,
                           train_dataloader=GTA5_dataloader,
                           validation_dataloader=cityscapes_test_dataloader,
                           epochs=EPOCHS,
                           device=device,
                           model_name="bisenet",
                           class_number=NC)



In [ ]:
save_results(model_bisenet_3_1, bisenet_result_3_1, "bisenet_adam_performance_metrics_3_1", height=CITYSCAPE_SIZE[0], width=CITYSCAPE_SIZE[1], iterations=1000, device=device)
plot_loss(bisenet_result_3_1, "BiSeNet", "step3_1_adam", "GTA5", "CityScapes")
plot_mIoU(bisenet_result_3_1, "BiSeNet", "step3_1_adam", "GTA5", "CityScapes")
plot_IoU(bisenet_result_3_1, "BiSeNet", "step3_1_adam", "GTA5", "CityScapes")
torch.save(model_bisenet_3_1.state_dict(), "./checkpoints/bisenet_adam_model_3_1.pth")



## 3.2 Bisenet with augmentation



In [ ]:
init_lr = 2.5e-4

best_score = 0
best_transform = ''

# Evaluate each augmentation strategy
for transform_name, transform_pipeline in augmentations.items():
    # Prepare datasets with the current transformation
    cityscapes_train_dataset = CityScapes('./Cityscapes', 'train', transform=transform_pipeline)
    cityscapes_test_dataset = CityScapes('./Cityscapes', 'val', transform=transform_pipeline)
    GTA5_dataset = GTA5('./GTA5', transform=transform_pipeline)

    # Create dataloaders
    cityscapes_train_dataloader = DataLoader(cityscapes_train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
    cityscapes_test_dataloader = DataLoader(cityscapes_test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
    GTA5_dataloader = DataLoader(GTA5_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
    
    # Initialize model and optimizer
    model_bisenet_3_2 = BiSeNet(19, 'resnet18').to(device)
    if device == 'cuda' and torch.cuda.device_count() > 1:
        model_bisenet_3_2 = torch.nn.DataParallel(model_bisenet_3_2).to(device)
    loss_fn = torch.nn.CrossEntropyLoss(ignore_index=255)
    optimizer = torch.optim.Adam(model_bisenet_3_2.parameters(), lr=init_lr)

    # Train the model
    bisenet_result3_2 = train(model=model_bisenet_3_2,
                              optimizer=optimizer,
                              loss_fn=loss_fn,
                              train_dataloader=GTA5_dataloader,
                              validation_dataloader=cityscapes_test_dataloader,
                              epochs=EPOCHS,
                              device=device,
                              model_name="bisenet",
                              class_number=NC)
    
    # Save and plot results
    save_results(model_bisenet_3_2, bisenet_result3_2, f"bisenet_sgd_performance_metrics_3_2_{transform_name}", height=CITYSCAPE_SIZE[0], width=CITYSCAPE_SIZE[1], iterations=100, device=device)
    plot_loss(bisenet_result3_2, "BiSeNet", f"step3_2_sgd_{transform_name}", "GTA5", "CityScapes")
    plot_mIoU(bisenet_result3_2, "BiSeNet", f"step3_2_sgd_{transform_name}", "GTA5", "CityScapes")
    plot_IoU(bisenet_result3_2, "BiSeNet", f"step3_2_sgd_{transform_name}", "GTA5", "CityScapes")
    
    # Save model checkpoint
    torch.save(model_bisenet_3_2.state_dict(), f"./checkpoints/bisenet_sgd_model_3_2_{transform_name}.pth")
    
    # Update best performing transformation
    if best_score < bisenet_result3_2[3][-1]:
        best_score = bisenet_result3_2[3][-1]
        best_transform = transform_name

best_transform


# 4.1 FDA

In [ ]:

GTA5_dataset = GTA5('./GTA5', transform=augmentations['transform2'], FDA = 0.01)
GTA5_dataloader = DataLoader(GTA5_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

model_bisenet_4_1_FDA = BiSeNet(19, 'resnet18').to(device)
if device == 'cuda' and torch.cuda.device_count() > 1:
    model_bisenet_4_1_FDA = torch.nn.DataParallel(model_bisenet_4_1_FDA).to(device)
init_lr = 2.5e-4 #0.0001
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=255)
optimizer = torch.optim.Adam(model_bisenet_4_1_FDA.parameters(), lr=init_lr)
bisenet_result4_1_FDA = train(model = model_bisenet_4_1_FDA,
                              optimizer = optimizer,
                              loss_fn = loss_fn,
                              train_dataloader = GTA5_dataloader,
                              validation_dataloader = cityscapes_test_dataloader,
                              epochs = EPOCHS,
                              device = device,
                              model_name = "bisenet",
                              class_number= NC)

    
save_results(model_bisenet_4_1_FDA, bisenet_result4_1_FDA, "bisenet_adam_performance_metrics_4_1_FDA7", height=CITYSCAPE_SIZE[0], width=CITYSCAPE_SIZE[1], iterations=100, device=device)
plot_loss(bisenet_result4_1_FDA, "BiSeNet", "step4_1_adam_FDA7", "GTA5", "CityScapes")
plot_mIoU(bisenet_result4_1_FDA, "BiSeNet", "step4_1_adam_FDA7", "GTA5", "CityScapes")
plot_IoU(bisenet_result4_1_FDA, "BiSeNet", "step4_1_adam_FDA7", "GTA5", "CityScapes")
torch.save(model_bisenet_4_1_FDA.state_dict(), f"./checkpoints/bisenet_adam_model_4_1_FDA7.pth")


# 4.2 DACS

In [ ]:

# GTA5_dataset = GTA5('./GTA5', transform=augmentations[best], FDA = 0.09)
GTA5_dataset = GTA5('./GTA5', transform=augmentations['transform2'])
GTA5_dataloader = DataLoader(GTA5_dataset, batch_size=2, shuffle=False, num_workers=NUM_WORKERS)
# GTA5_dataset = GTA5('./GTA5', transform=augmentations[best_transform], FDA = 0.09)

# GTA5_dataloader = DataLoader(GTA5_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
model_bisenet_4_2_DACS = BiSeNet(NC, 'resnet18').to(device)
if device == 'cuda' and torch.cuda.device_count() > 1:
    model_bisenet_4_2_DACS = torch.nn.DataParallel(model_bisenet_4_2_DACS).to(device)
init_lr = 2.5e-4
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=255)
optimizer = torch.optim.Adam(model_bisenet_4_2_DACS.parameters(), lr=init_lr)

bisenet_result4_2_DACS = train(model = model_bisenet_4_2_DACS,
                               optimizer = optimizer,
                               loss_fn = loss_fn,
                               train_dataloader= GTA5_dataloader,
                               validation_dataloader=cityscapes_test_dataloader,
                               target_dataloader=cityscapes_train_dataloader,
                               epochs=EPOCHS,
                               device = device,
                               model_name='bisenet',
                               )
save_results(model_bisenet_4_2_DACS, bisenet_result4_2_DACS, "bisenet_sgd_performance_metrics_4_2_DACS", height=CITYSCAPE_SIZE[0], width=CITYSCAPE_SIZE[1], iterations=100, device=device)
plot_loss(bisenet_result4_2_DACS, "BiSeNet_sgd", "step4_2_DACS", "GTA5 + CityScapes (DACS)", "CityScapes")
plot_mIoU(bisenet_result4_2_DACS, "BiSeNet_sgd", "step4_2_DACS", "GTA5 + CityScapes (DACS)", "CityScapes")
plot_IoU(bisenet_result4_2_DACS, "BiSeNet_sgd", "step4_2_DACS", "GTA5 + CityScapes (DACS)", "CityScapes")

torch.save(model_bisenet_4_2_DACS.state_dict(), "./checkpoints/bisenet_sgd_model_4_2_DACS.pth")
